In [1]:
library(tidyverse)
library(ggplot2)
library(patchwork)
library(geneBasisR)
library(Seurat)
library(wesanderson)
library(ggcorrplot)
library(ggpubr)
library(SingleCellExperiment)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Attaching SeuratObject

Attaching sp

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, co

In [2]:
run_name = 'litgenes_all_n1xo_nosct__select_batch.cell_class___eval.neuron_1xo__predict.polar_label__eval_batch.none'
# lib_stat_path = paste0('_targets/objects/lib_stat_batched_litgenes_all_n1xo_nosct__select_batch.none___eval.other__predict.polar_label__eval_batch.none')
lib_stat_path = paste0('_targets/objects/lib_stat_batched_reranked_', run_name)
lib_stat = qs::qread(lib_stat_path)

In [3]:
genes_stat_path = paste0('_targets/objects/genes_stat_reranked_', run_name)
genes_stat = qs::qread(genes_stat_path)

In [4]:
run_name_other = 'litgenes_all_n1xo_nosct__select_batch.cell_class___eval.other__predict.polar_label__eval_batch.none'
genes_stat_path_other = paste0('_targets/objects/genes_stat_reranked_', run_name_other)
genes_stat_other = qs::qread(genes_stat_path_other)

In [5]:
genes_stat_path == genes_stat_path_other

[1] FALSE

In [6]:
(genes_stat_other == genes_stat) %>% head

rank,gene,mean_gene_score,rank_orig
TRUE,FALSE,FALSE,FALSE
TRUE,FALSE,FALSE,FALSE
TRUE,FALSE,FALSE,FALSE
TRUE,FALSE,FALSE,FALSE
TRUE,FALSE,FALSE,FALSE
TRUE,FALSE,FALSE,FALSE


In [7]:
litgenes = c("Pdgfra", # (OPC)
               "Bmp4", # (NFOL)
               "Plp1", # (MOL)
               "Aqp4", # (Astro)
               "Rax", # (tany)
               "Agrp",
               "Pomc",
               "Htr3b",
               "Lef1",
               "Lmx1a")

In [8]:
genes_stat_both = full_join(genes_stat, genes_stat_other, by = "gene", suffix = c(".neuron", ".other"))
genes_stat_both %>% head

rank.neuron,gene,mean_gene_score.neuron,rank_orig.neuron,rank.other,mean_gene_score.other,rank_orig.other
<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>
1,Lmx1a,1.114295,10,93,NA,10
2,Htr3b,1.095357,8,4,1.5406753,8
3,Lef1,1.085229,9,92,NA,9
4,Il1rapl2,1.029219,17,7,1.3099371,17
5,Col25a1,1.027266,13,16,1.1422594,13
6,Trpm3,1.000497,22,46,0.9924366,22


In [9]:
rank_cutoff = 70
drop_genes_70 = genes_stat_both %>%
filter(((rank.neuron > rank_cutoff) & (rank.other > rank_cutoff))) %>%
filter(!(gene %in% litgenes)) %>%
pull(gene)

In [10]:
rank_cutoff_neuron = 70
rank_cutoff_other = 70
drop_genes_70 = genes_stat_both %>%
filter(((rank.neuron > rank_cutoff_neuron) & (rank.other > rank_cutoff_other))) %>%
filter(!(gene %in% litgenes)) %>%
pull(gene)

In [11]:
100-length(drop_genes_70)

[1] 85

In [14]:
genes_stat_85 = genes_stat_other %>%
filter(!(gene %in% drop_genes_70))

qs::qsave(genes_stat_85, "genes_stat_85.qs")

In [15]:
genes_stat_85

rank,gene,mean_gene_score,rank_orig
<int>,<chr>,<dbl>,<int>
1,Nrg3,2.206128,11
2,Plp1,1.559852,3
3,Aqp4,1.545171,4
4,Htr3b,1.540675,8
5,Rax,1.359362,5
6,Rbfox1,1.322021,53
7,Il1rapl2,1.309937,17
8,Robo1,1.301079,30
9,Kcnip4,1.266857,18
